In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/notebooks/sensorium_2023')

In [3]:
!pip install -q git+https://github.com/sinzlab/nnfabrik.git
!pip install -q git+https://github.com/sinzlab/neuralpredictors

!pip install -q deeplake
!pip install -q transformers --upgrade
!pip install -q  av
!pip install -q decord

#bits and bytes
# Install latest bitsandbytes & transformers, accelerate from source
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
# Other requirements for the demo
!pip install gradio
!pip install sentencepiece

!pip install -q -U accelerate
!pip install -q -U bitsandbytes
!pip install -q -i https://test.pypi.org/simple/ bitsandbytes

!pip install wandb


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.25.91 requires docutils<0.17,>=0.10, but you have docutils 0.20.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.4.0 requires dill<0.3.6, but you have dill 0.3.7 which is incompatible.
awscli 1.25.91 requires botocore==1.27.90, but you have botocore 1.31.17 which is incompatible.
awscli 1.25.91 requires docutils<0.17,>=0.10, but you have docutils 0.20.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.4.0 requires dill<0.3.6, but you have dill 0.3.7 which is incompatible.
ERROR: pip's depe

In [4]:
!pip install plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 94.6 MB/s eta 0:00:0000:0100:01


In [3]:
import deeplake
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib
from IPython.display import HTML
from matplotlib import animation
import imageio
import os
import numpy as np
import plotly.express as px
from functools import partial

from torchvision.transforms import Compose

# transformers stuff

import av
import numpy as np
import torch

import transformers

from transformers import VivitImageProcessor, VivitModel, VivitForVideoClassification
from huggingface_hub import hf_hub_download

from decord import VideoReader

import time

from transformers import BitsAndBytesConfig

import accelerate
import bitsandbytes


/usr/local/lib/python3.9/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [4]:
from pathlib import Path
def load_yaml(file_path: str):
    """
    Load YAML file into a dictionary. 
    Args: file_path (str): 
    Path to the YAML file to be loaded
    """    
    file_path = Path(file_path)
    with file_path.open('r') as f:
        data = yaml.safe_load(f)
    return data

In [5]:
import wandb
import random
import yaml

# start a new wandb run to track this script
# config = load_yaml("configs/test.yaml")
# wandb_project = config.pop("project")
wandb.init(
    # set the wandb project where this run will be logged
    # project=wandb_project,
    # track hyperparameters and run metadata
    # config= config
       
)
# config = wandb.config 
# checkpoint_dir = Path(config.checkpoint_root)/config.name
checkpoint_dir = Path('checkpoints')

wandb: Currently logged in as: rogershijin. Use `wandb login --relogin` to force relogin


In [6]:

paths = [
'/storage/sensorium/data/dynamic29515-10-12-Video-9b4f6a1a067fe51e15306b9628efea20/',
'/storage/sensorium/data/dynamic29623-4-9-Video-9b4f6a1a067fe51e15306b9628efea20/',
'/storage/sensorium/data/dynamic29647-19-8-Video-9b4f6a1a067fe51e15306b9628efea20/',
'/storage/sensorium/data/dynamic29712-5-9-Video-9b4f6a1a067fe51e15306b9628efea20/',
'/storage/sensorium/data/dynamic29755-2-8-Video-9b4f6a1a067fe51e15306b9628efea20/'
]

'''
mice = [
    'dynamic29156-11-10-Video-8744edeac3b4d1ce16b680916b5267ce',
    'dynamic29228-2-10-Video-8744edeac3b4d1ce16b680916b5267ce',
    'dynamic29234-6-9-Video-8744edeac3b4d1ce16b680916b5267ce',
    'dynamic29513-3-5-Video-8744edeac3b4d1ce16b680916b5267ce',
    'dynamic29514-2-9-Video-8744edeac3b4d1ce16b680916b5267ce',
]

mice = [
    'dynamic29755-2-8-Video-9b4f6a1a067fe51e15306b9628efea20',
    'dynamic29515-10-12-Video-9b4f6a1a067fe51e15306b9628efea20',
    'dynamic29623-4-9-Video-9b4f6a1a067fe51e15306b9628efea20',
    'dynamic29647-19-8-Video-9b4f6a1a067fe51e15306b9628efea20',
    'dynamic29712-5-9-Video-9b4f6a1a067fe51e15306b9628efea20',
]
'''

"\nmice = [\n    'dynamic29156-11-10-Video-8744edeac3b4d1ce16b680916b5267ce',\n    'dynamic29228-2-10-Video-8744edeac3b4d1ce16b680916b5267ce',\n    'dynamic29234-6-9-Video-8744edeac3b4d1ce16b680916b5267ce',\n    'dynamic29513-3-5-Video-8744edeac3b4d1ce16b680916b5267ce',\n    'dynamic29514-2-9-Video-8744edeac3b4d1ce16b680916b5267ce',\n]\n\nmice = [\n    'dynamic29755-2-8-Video-9b4f6a1a067fe51e15306b9628efea20',\n    'dynamic29515-10-12-Video-9b4f6a1a067fe51e15306b9628efea20',\n    'dynamic29623-4-9-Video-9b4f6a1a067fe51e15306b9628efea20',\n    'dynamic29647-19-8-Video-9b4f6a1a067fe51e15306b9628efea20',\n    'dynamic29712-5-9-Video-9b4f6a1a067fe51e15306b9628efea20',\n]\n"

In [7]:
seed = 42

In [8]:
!pip install -q datajoint

In [9]:
# Uncomment if imports of sensorium do not work 
# !ln -s ../sensorium

import sys
sys.path.append('/notebooks/teaching/sandra/sensorium_2023')
sys.path.append('/notebooks/sensorium_2023')

import torch
from nnfabrik.utility.nn_helpers import set_random_seed
set_random_seed(seed)

from sensorium.datasets.mouse_video_loaders import mouse_video_loader
from sensorium.utility.scores import get_correlations
from nnfabrik.builder import get_trainer
from sensorium.models.make_model import make_video_model

device = 'cuda'

In [38]:

# dataset = deeplake.load(f'/storage/sensorium/data/{mice[0]}/data')

# dataset = deeplake.load(f'hub://sinzlab/Sensorium_2023_{mice[0]}_train')

print("Loading data..")
data_loaders = mouse_video_loader(
    paths=paths,
    batch_size=1,
    scale=1,
    max_frame=None,
    frames=64, # frames has to be > 50. If it fits on your gpu, we recommend 150
    offset=-1,
    include_behavior=True,
    include_pupil_centers=True,
    cuda=device!='cpu',
    to_cut=False,
)
print('Data loaded')


# if DEV:
#     dataset = dataset[:10]
# dataset.summary()

Loading data..
Data loaded


In [39]:
data_loaders.keys()

dict_keys(['live_test_bonus', 'train', 'live_test_main', 'final_test_bonus', 'final_test_main', 'oracle'])

In [40]:
MICE = sorted(list(data_loaders['train'].keys()))
    

In [41]:
data_loaders_transpose = {}

for stage, stage_dataloaders in data_loaders.items():
    for mouse, dataloader in stage_dataloaders.items():
        if mouse not in data_loaders_transpose:
            data_loaders_transpose[mouse] = {stage: dataloader}
        else:
            data_loaders_transpose[mouse][stage] = dataloader

MOUSE_SIZES = {}
for mouse in MICE:
    batch = next(iter(data_loaders_transpose[mouse]['train']))
    MOUSE_SIZES[mouse] = batch.responses.shape[1]
MOUSE_SIZES

data_loaders_transpose

{'dynamic29515-10-12-Video-9b4f6a1a067fe51e15306b9628efea20': {'live_test_bonus': <torch.utils.data.dataloader.DataLoader at 0x7f45142aa880>,
  'train': <torch.utils.data.dataloader.DataLoader at 0x7f45142aad30>,
  'live_test_main': <torch.utils.data.dataloader.DataLoader at 0x7f45142aad60>,
  'final_test_bonus': <torch.utils.data.dataloader.DataLoader at 0x7f45142aa340>,
  'final_test_main': <torch.utils.data.dataloader.DataLoader at 0x7f45142aa4c0>,
  'oracle': <torch.utils.data.dataloader.DataLoader at 0x7f45142aa7c0>},
 'dynamic29623-4-9-Video-9b4f6a1a067fe51e15306b9628efea20': {'live_test_bonus': <torch.utils.data.dataloader.DataLoader at 0x7f4514432670>,
  'train': <torch.utils.data.dataloader.DataLoader at 0x7f45143144f0>,
  'live_test_main': <torch.utils.data.dataloader.DataLoader at 0x7f4514314cd0>,
  'final_test_bonus': <torch.utils.data.dataloader.DataLoader at 0x7f4514314340>,
  'final_test_main': <torch.utils.data.dataloader.DataLoader at 0x7f4514314e80>,
  'oracle': <torc

In [42]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [43]:
from bitsandbytes.nn import Linear4bit
from torch.nn import Linear
import torch
import torch.nn as nn
from transformers import VivitImageProcessor, VivitModel, VivitForVideoClassification
from transformers.modeling_outputs import ImageClassifierOutput
        

class Swappable(nn.Module):
    
    def __init__(self, named_layers: dict[str, nn.Module]):
        super().__init__()
        self.named_layers = named_layers
        self._named_layers = nn.ModuleList(named_layers.values())
    
    def forward(self, x, name):
        return self.named_layers[name](x)

def pad_to_nearest_multiple(tensor, target_multiple):
    """Pads a tensor to the nearest multiple along the second dimension.
    
    Args:
        tensor (torch.Tensor): Input tensor to pad.
        target_multiple (int): The target multiple to pad the second dimension to.
        
    Returns:
        torch.Tensor: Padded tensor.
    """
    batch_size, seq_len, hidden_size = tensor.size()
    
    # Calculate the padding size needed to reach the nearest multiple
    padding_size = (target_multiple - (seq_len % target_multiple)) % target_multiple
    
    # Calculate the padding for the left and right sides
    left_padding = padding_size // 2
    right_padding = padding_size - left_padding
    
    # Create padding tuple
    padding_tuple = (0, 0, left_padding, right_padding)
    
    # Apply padding
    padded_tensor = torch.nn.functional.pad(tensor, padding_tuple, 'constant', 0)
    
    return padded_tensor

class Reducer(nn.Module):
    
    def __init__(self, input_seq_len=3137, target_seq_len=32, hidden_dim=768):
        super().__init__()
        self.target_seq_len = target_seq_len
        padded_len = input_seq_len if input_seq_len % target_seq_len  == 0 else (input_seq_len // target_seq_len + 1) * 32
        stride = padded_len // target_seq_len
        kernel_size = stride
        self.conv1d = nn.Conv1d(hidden_dim, hidden_dim, kernel_size, stride=stride)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        '''
        x has size batch_size x seq_len x hidden_dim
        '''
        x = pad_to_nearest_multiple(x, self.target_seq_len)
        print('x', x)
        x = x.permute(0, 2, 1)
        x = self.relu(x)
        x = self.conv1d(x)
        x = self.relu(x)
        x = x.permute(0, 2, 1)
        return x

class ReducerClassifier(nn.Module):
    
    def __init__(self, out_dim, input_seq_len=3137, target_seq_len=32, hidden_dim=768):
        super().__init__()
        self.reducer = Reducer(input_seq_len=input_seq_len, target_seq_len=target_seq_len, hidden_dim=hidden_dim)
        self.linear = Linear(hidden_dim, out_dim)
        
    def forward(self, x):
        x = self.reducer(x)
        x = self.linear(x)
        return x
    
    
class SensoriumVivitForVideoClassification(VivitForVideoClassification):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # self.reducer = Re
        # self.reducer = Reducer()
    
    def forward(
        self,
        pixel_values=None,
        mouse=None,
        inputs_embeds=None,
        input_ids=None,
        attention_mask=None,
        head_mask=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.vivit(
            pixel_values,
            head_mask=head_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

#         sequence_output = outputs[0]

#         logits = self.classifier(sequence_output[:, 0, :])

#         reduced = self.reducer(outputs.last_hidden_state)
#         print('reduced', reduced)

#         logits = self.classifier(reduced, mouse)
#         print('logits', logits)
#         reduced = self.reducer(outputs.last_hidden_state)
#         print('reduced', reduced)

        logits = self.classifier(outputs.last_hidden_state[:, 0, :], mouse)
        

        loss = None
        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return ImageClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [44]:
WINDOW_SIZE = 32

image_processor = VivitImageProcessor.from_pretrained("google/vivit-b-16x2-kinetics400")

model = SensoriumVivitForVideoClassification.from_pretrained("google/vivit-b-16x2-kinetics400", quantization_config=bnb_config)

# model.classifier= Linear(768, 38000, compute_dtype=torch.bfloat16).cuda()


# model = VivitWrapper(model)

model.classifier = Swappable({mouse: Linear(768, output_size*WINDOW_SIZE) for mouse, output_size in MOUSE_SIZES.items()})

In [45]:
model

SensoriumVivitForVideoClassification(
  (vivit): VivitModel(
    (embeddings): VivitEmbeddings(
      (patch_embeddings): VivitTubeletEmbeddings(
        (projection): Conv3d(3, 768, kernel_size=(2, 16, 16), stride=(2, 16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): VivitEncoder(
      (layer): ModuleList(
        (0-11): 12 x VivitLayer(
          (attention): VivitAttention(
            (attention): VivitSelfAttention(
              (query): Linear4bit(in_features=768, out_features=768, bias=True)
              (key): Linear4bit(in_features=768, out_features=768, bias=True)
              (value): Linear4bit(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): VivitSelfOutput(
              (dense): Linear4bit(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): Vivit

In [46]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [47]:
# black magic to make sure we train the classifiers
# def make_module_require_grad(module, input, output):
#     output.requires_grad_(True)


In [48]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

print_trainable_parameters(model)

trainable params: 0 || all params: 1031335744 || trainable%: 0.0


In [49]:
model

SensoriumVivitForVideoClassification(
  (vivit): VivitModel(
    (embeddings): VivitEmbeddings(
      (patch_embeddings): VivitTubeletEmbeddings(
        (projection): Conv3d(3, 768, kernel_size=(2, 16, 16), stride=(2, 16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): VivitEncoder(
      (layer): ModuleList(
        (0-11): 12 x VivitLayer(
          (attention): VivitAttention(
            (attention): VivitSelfAttention(
              (query): Linear4bit(in_features=768, out_features=768, bias=True)
              (key): Linear4bit(in_features=768, out_features=768, bias=True)
              (value): Linear4bit(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): VivitSelfOutput(
              (dense): Linear4bit(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): Vivit

In [50]:
# for name, param in model.named_parameters():
#     print(f"Parameter name: {name}, Dtype: {param.dtype}")

In [51]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query", "key", "value"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"
)

model = get_peft_model(model, config)
# train the classifier layer
for param in model.classifier.parameters():
    # classifier.register_forward_hook(make_module_require_grad)
    param.requires_grad_(True)
# for param in model.reducer.parameters():
#     # classifier.register_forward_hook(make_module_require_grad)
#     param.requires_grad_(True)
print_trainable_parameters(model)
model.to('cuda:0')

trainable params: 1970755712 || all params: 2016934784 || trainable%: 97.71043306078458


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): SensoriumVivitForVideoClassification(
      (vivit): VivitModel(
        (embeddings): VivitEmbeddings(
          (patch_embeddings): VivitTubeletEmbeddings(
            (projection): Conv3d(3, 768, kernel_size=(2, 16, 16), stride=(2, 16, 16))
          )
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (encoder): VivitEncoder(
          (layer): ModuleList(
            (0-11): 12 x VivitLayer(
              (attention): VivitAttention(
                (attention): VivitSelfAttention(
                  (query): Linear4bit(
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_feature

In [52]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): SensoriumVivitForVideoClassification(
      (vivit): VivitModel(
        (embeddings): VivitEmbeddings(
          (patch_embeddings): VivitTubeletEmbeddings(
            (projection): Conv3d(3, 768, kernel_size=(2, 16, 16), stride=(2, 16, 16))
          )
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (encoder): VivitEncoder(
          (layer): ModuleList(
            (0-11): 12 x VivitLayer(
              (attention): VivitAttention(
                (attention): VivitSelfAttention(
                  (query): Linear4bit(
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_feature

In [53]:
# make sure we're training the classifiers
assert next(model.classifier.parameters()).requires_grad
assert next(model.classifier.parameters()).device == model.device

In [54]:
optimizer = bitsandbytes.optim.AdamW(model.parameters(),lr=3e-5,optim_bits=8, is_paged=True)

In [55]:
from torch.nn import MSELoss
loss_fn = MSELoss()

In [56]:
# train and validation

In [57]:
# find longest response mouse
max_length = 0
max_mouse = None
mice_lengths = []
mice_lengths_train = []
for stage, stage_dataloaders in data_loaders.items():
    for mouse, dataloader in stage_dataloaders.items():
        for batch in dataloader:
            length = batch.responses.shape[1]
            mice_lengths.append((length, stage, mouse))
            if 'train' in stage:
                mice_lengths_train.append((length, stage, mouse))
            if length > max_length:
                max_length = length
                max_mouse = (stage, mouse)
            break
print(max_length, max_mouse)
print(sorted(mice_lengths, key=lambda elt: elt[0], reverse=True))
print(sorted(mice_lengths_train, key=lambda elt: elt[0], reverse=True))

8202 ('live_test_bonus', 'dynamic29647-19-8-Video-9b4f6a1a067fe51e15306b9628efea20')
[(8202, 'live_test_bonus', 'dynamic29647-19-8-Video-9b4f6a1a067fe51e15306b9628efea20'), (8202, 'train', 'dynamic29647-19-8-Video-9b4f6a1a067fe51e15306b9628efea20'), (8202, 'live_test_main', 'dynamic29647-19-8-Video-9b4f6a1a067fe51e15306b9628efea20'), (8202, 'final_test_bonus', 'dynamic29647-19-8-Video-9b4f6a1a067fe51e15306b9628efea20'), (8202, 'final_test_main', 'dynamic29647-19-8-Video-9b4f6a1a067fe51e15306b9628efea20'), (8202, 'oracle', 'dynamic29647-19-8-Video-9b4f6a1a067fe51e15306b9628efea20'), (8122, 'live_test_bonus', 'dynamic29755-2-8-Video-9b4f6a1a067fe51e15306b9628efea20'), (8122, 'train', 'dynamic29755-2-8-Video-9b4f6a1a067fe51e15306b9628efea20'), (8122, 'live_test_main', 'dynamic29755-2-8-Video-9b4f6a1a067fe51e15306b9628efea20'), (8122, 'final_test_bonus', 'dynamic29755-2-8-Video-9b4f6a1a067fe51e15306b9628efea20'), (8122, 'final_test_main', 'dynamic29755-2-8-Video-9b4f6a1a067fe51e15306b9628e

In [58]:
def concatenate_dataloaders(dataloaders, verbose=True):
    iterators = {mouse: iter(loader) for mouse,loader in dataloaders.items()}
    total = sum([len(loader) for loader in dataloaders.values()])
    if verbose:
        print('total from concatenate loaders', total)
    count = 0
    while True:
        for mouse, iterator in iterators.items():
            if count > total:
                return
            try:
                yield mouse, next(iterator)
                count += 1
            except StopIteration:
                return

In [59]:
model = model.to('cuda:0')

In [ ]:
import math 
from datetime import datetime
from tqdm.notebook import tqdm

DEBUG = False


best_val_loss = math.inf
save_period = 1
val_period = 1
n_epochs = 3
checkpoint_dir = Path("checkpoints")/str(datetime.now()).replace(" ", "_")
checkpoint_dir.mkdir(parents = True)
train_total = sum([len(loader) for loader in data_loaders['train'].values()])
print('total batches', train_total)
steps = 0
for epoch in range(1,n_epochs+1):
    print('EPOCH', epoch)
    model.train()
    for mouse, batch in tqdm(concatenate_dataloaders(data_loaders['train']), total=train_total):
        # model.classifier.to('cpu')
        # model.classifier = classifiers[mouse].to('cuda:0')
        # for batch in tqdm(data_loaders['train'][mouse]):
            # print(batch.videos)
        start = random.randint(50, 150-64)
        videos = batch.videos[:,:,start:start+64:2,:,:].permute(0,2,3,4,1)
        video_range = videos.max()-videos.min()
        videos -= videos.max() - video_range/2
        videos /= video_range + 0.01
        videos += .5
        videos = videos.squeeze()
        videos = image_processor(list(videos), return_tensors="pt")
        videos['pixel_values'] = videos['pixel_values'].cuda()
        responses = batch.responses[:,:,start:start+64:2].cuda()

        optimizer.zero_grad()
        output = model(**videos, mouse=mouse)
        # print(output.logits)
        train_loss= loss_fn(output.logits.reshape(-1, MOUSE_SIZES[mouse], WINDOW_SIZE), torch.log(torch.nn.functional.relu(responses)+1)) #torch.log(batch['responses'])
        # print(train_loss)
        wandb.log({'train_loss':train_loss.item()})
        train_loss.backward()
        optimizer.step()
        steps += 1
        if DEBUG:
            break
        if steps % 500 == 0:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, checkpoint_dir/f"step_{steps}.pt")
            
    
    if steps % save_period == 0:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, checkpoint_dir/f"{epoch}.pt")

    if epoch%val_period==0:
        total_val_loss = 0

        for mouse in data_loaders['oracle']:
            # model.classifier.to('cpu')
            # model.classifier = classifiers[mouse].to('cuda:0')
            model.eval()
            for batch in data_loaders['oracle'][mouse]:
                start = random.randint(50, 150-64)
                videos = batch.videos[:,:,start:start+64:2,:,:].permute(0,2,3,4,1)
                video_range = videos.max()-videos.min()
                videos -= videos.max() - video_range/2
                videos /= video_range + 0.01
                videos += .5
                videos = videos.squeeze()
                videos = image_processor(list(videos), return_tensors="pt")
                videos['pixel_values'] = videos['pixel_values'].cuda()
                responses = batch.responses[:,:,start:start+64:2].cuda()

                with torch.no_grad():
                    output = model(**videos, mouse=mouse)
                    val_loss= loss_fn(output.logits.reshape(-1, MOUSE_SIZES[mouse], WINDOW_SIZE), torch.log(torch.nn.functional.relu(responses)+1)) #torch.log(batch['responses'])          
                total_val_loss+=val_loss
                if DEBUG:
                    break
            wandb.log({'val_loss':total_val_loss.item()})
            print("train_loss", train_loss.item())
            print("val loss:", total_val_loss.item())
            if total_val_loss < best_val_loss:
                best_val_loss = total_val_loss
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'val_loss': total_val_loss,
                }, checkpoint_dir/"best.pt")
                print(f"save checkpoint to {checkpoint_dir/'best.pt'}")

total batches 1744
EPOCH 1


  0%|          | 0/1744 [00:00<?, ?it/s]

total from concatenate loaders 1744


/usr/local/lib/python3.9/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


In [35]:
steps

611

In [36]:
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, checkpoint_dir/f"step_{steps}.pt")

In [37]:
checkpoint_dir

PosixPath('checkpoints/2023-10-16_09:14:41.068286')

In [38]:
! cp -r {str(checkpoint_dir)} /storage/

In [ ]:
from bitsandbytes.nn import Linear4bit
from torch.nn import Linear
from transformers import VivitImageProcessor, VivitModel, VivitForVideoClassification

checkpoint_dir = None
checkpoint = torch.load(checkpoint_dir/'best.pt')

image_processor = VivitImageProcessor.from_pretrained("google/vivit-b-16x2-kinetics400")
model = VivitForVideoClassification.from_pretrained("google/vivit-b-16x2-kinetics400", quantization_config=bnb_config)
# model.classifier= Linear(768, 38000, compute_dtype=torch.bfloat16).cuda()
model.classifier= Linear(768, 7495*WINDOW_SIZE).cuda()
# model = VivitWrapper(model)
inputs = image_processor(list(video), return_tensors="pt")
inputs['pixel_values'] = inputs['pixel_values'].repeat(2,1,1,1,1)
print(inputs['pixel_values'].shape)


from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query", "key", "value"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
model.eval()
total_test_loss = 0
predictions = []
for batch in dataloaders['oracle']:
    with torch.no_grad():
        output = model(**batch['videos'])
        predictions.append(output.logits.reshape(-1,7495,WINDOW_SIZE).argmax(dim=1))
        test_loss= loss_fn(output.logits.reshape(-1,7495,WINDOW_SIZE), torch.log(torch.nn.functional.relu(batch['responses'])+1)) #torch.log(batch['responses'])          
    total_test_loss+=test_loss
wandb.log({'test_loss':total_test_loss})
print("test loss")
print(total_test_loss)